# Машинное обучение, ФКН ВШЭ

## Практическое задание 7. Градиентный бустинг ~~своими руками~~

### Общая информация
Дата выдачи: 07.12.2018

Мягкий дедлайн: 05:59MSK 18.12.2018

Жесткий дедлайн: 05:59MSK 19.12.2018

### Оценивание и штрафы
Каждая из задач имеет определенную «стоимость» (указана в скобках около задачи). Максимально допустимая оценка за работу — 10 баллов.

Сдавать задание после указанного срока сдачи нельзя. При выставлении неполного балла за задание в связи с наличием ошибок на усмотрение проверяющего предусмотрена возможность исправить работу на указанных в ответном письме условиях.

Задание выполняется самостоятельно. «Похожие» решения считаются плагиатом и все задействованные студенты (в том числе те, у кого списали) не могут получить за него больше 0 баллов (подробнее о плагиате см. на странице курса). Если вы нашли решение какого-то из заданий (или его часть) в открытом источнике, необходимо указать ссылку на этот источник в отдельном блоке в конце вашей работы (скорее всего вы будете не единственным, кто это нашел, поэтому чтобы исключить подозрение в плагиате, необходима ссылка на источник).

Неэффективная реализация кода может негативно отразиться на оценке.

### Формат сдачи
Задания сдаются через систему anytask. Посылка должна содержать:
* Ноутбук homework-practice-07-Username.ipynb

Username — ваша фамилия и имя на латинице именно в таком порядке

__Задание 1. (0.5 балла)__

Мы будем использовать данные соревнования [Home Credit Default Risk](https://www.kaggle.com/c/home-credit-default-risk/data). 
* Загрузите таблицу application_train.csv;
* Запишите в Y столбец с целевой переменной;
* Удалите ненужные столбцы (для этого воспользуйтесь описанием);
* Определите тип столбцов и заполните пропуски - стратегия произвольная;
* Разбейте выборку в соотношении 70:30 с random_state=0.

Так как в данных значительный дисбаланс классов, в качестве метрики качества везде будем использовать площадь под precision-recall кривой.

In [1]:
import pandas as pd
df = pd.read_csv('application_train.csv')
df.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [2]:
y = df['TARGET']
df = df.drop(columns=['TARGET', 'SK_ID_CURR'])
df.head()

,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,351000.0,Unaccompanied,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,1129500.0,Family,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,135000.0,Unaccompanied,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,297000.0,Unaccompanied,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,513000.0,Unaccompanied,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [3]:
import numpy as np
nulls = df.isnull().sum(axis=0)
strs = df.dtypes[df.dtypes == object].axes[0]
ints = df.dtypes[df.dtypes == 'int64'].axes[0]
floats = df.dtypes[df.dtypes == 'float64'].axes[0]

In [4]:
nulls[strs]

NAME_CONTRACT_TYPE                 0
CODE_GENDER                        0
FLAG_OWN_CAR                       0
FLAG_OWN_REALTY                    0
NAME_TYPE_SUITE                 1292
NAME_INCOME_TYPE                   0
NAME_EDUCATION_TYPE                0
NAME_FAMILY_STATUS                 0
NAME_HOUSING_TYPE                  0
OCCUPATION_TYPE                96391
WEEKDAY_APPR_PROCESS_START         0
ORGANIZATION_TYPE                  0
FONDKAPREMONT_MODE            210295
HOUSETYPE_MODE                154297
WALLSMATERIAL_MODE            156341
EMERGENCYSTATE_MODE           145755
dtype: int64

In [5]:
for s in strs:
    mf = df[s][df[s].isnull() == False].value_counts().idxmax()
    df[s][df[s].isnull()] = mf

/usr/lib/python3/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [6]:
nulls[ints]

CNT_CHILDREN                   0
DAYS_BIRTH                     0
DAYS_EMPLOYED                  0
DAYS_ID_PUBLISH                0
FLAG_MOBIL                     0
FLAG_EMP_PHONE                 0
FLAG_WORK_PHONE                0
FLAG_CONT_MOBILE               0
FLAG_PHONE                     0
FLAG_EMAIL                     0
REGION_RATING_CLIENT           0
REGION_RATING_CLIENT_W_CITY    0
HOUR_APPR_PROCESS_START        0
REG_REGION_NOT_LIVE_REGION     0
REG_REGION_NOT_WORK_REGION     0
LIVE_REGION_NOT_WORK_REGION    0
REG_CITY_NOT_LIVE_CITY         0
REG_CITY_NOT_WORK_CITY         0
LIVE_CITY_NOT_WORK_CITY        0
FLAG_DOCUMENT_2                0
FLAG_DOCUMENT_3                0
FLAG_DOCUMENT_4                0
FLAG_DOCUMENT_5                0
FLAG_DOCUMENT_6                0
FLAG_DOCUMENT_7                0
FLAG_DOCUMENT_8                0
FLAG_DOCUMENT_9                0
FLAG_DOCUMENT_10               0
FLAG_DOCUMENT_11               0
FLAG_DOCUMENT_12               0
FLAG_DOCUM

In [7]:
for f in floats:
    mean = df[f][df[f].isnull() == False].mean()
    df[f][df[f].isnull()] = mean

/usr/lib/python3/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [8]:
df.isnull().sum().sum()

0

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=0.3, random_state=0)

__Задание 2. (1.5 балла)__

Обучите реализации градиентного бустинга LightGBM и Catboost на вещественных признаках без подбора параметров. 
Почему получилась заметная разница в качестве? 

В этом и последующих экспериментах необходимо измерять время обучения моделей.

In [10]:
import lightgbm as lgbm
import catboost as cb

In [11]:
lgbmc = lgbm.LGBMClassifier()
%time
lgbmc.fit(X_train[floats], y_train)

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 6.44 µs


LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
        importance_type='split', learning_rate=0.1, max_depth=-1,
        min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
        n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
        random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
        subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [44]:
from sklearn.metrics import roc_auc_score
lgbm_pred = lgbmc.predict_proba(X_test[floats]).T[1]
roc_auc_score(y_test, lgbm_pred)

0.7442510162068767

In [20]:
cbc = cb.CatBoostClassifier()
%time
cbc.fit(X_train[floats], y_train)

CPU times: user 5 µs, sys: 0 ns, total: 5 µs
Wall time: 8.34 µs
0:	learn: 0.6162436	total: 161ms	remaining: 2m 41s
1:	learn: 0.5539626	total: 250ms	remaining: 2m 4s
2:	learn: 0.5050184	total: 343ms	remaining: 1m 54s
3:	learn: 0.4610355	total: 446ms	remaining: 1m 51s
4:	learn: 0.4268145	total: 573ms	remaining: 1m 54s
5:	learn: 0.3967463	total: 695ms	remaining: 1m 55s
6:	learn: 0.3748661	total: 812ms	remaining: 1m 55s
7:	learn: 0.3570012	total: 923ms	remaining: 1m 54s
8:	learn: 0.3405031	total: 1.05s	remaining: 1m 55s
9:	learn: 0.3271964	total: 1.15s	remaining: 1m 53s
10:	learn: 0.3160649	total: 1.26s	remaining: 1m 53s
11:	learn: 0.3074647	total: 1.37s	remaining: 1m 52s
12:	learn: 0.2988738	total: 1.51s	remaining: 1m 54s
13:	learn: 0.2926765	total: 1.63s	remaining: 1m 54s
14:	learn: 0.2872014	total: 1.74s	remaining: 1m 54s
15:	learn: 0.2829847	total: 1.85s	remaining: 1m 53s
16:	learn: 0.2795198	total: 1.97s	remaining: 1m 53s
17:	learn: 0.2761312	total: 2.08s	remaining: 1m 53s
18:	learn: 

156:	learn: 0.2491644	total: 18.5s	remaining: 1m 39s
157:	learn: 0.2491441	total: 18.6s	remaining: 1m 39s
158:	learn: 0.2491168	total: 18.8s	remaining: 1m 39s
159:	learn: 0.2490942	total: 18.9s	remaining: 1m 39s
160:	learn: 0.2490663	total: 19s	remaining: 1m 39s
161:	learn: 0.2490504	total: 19.1s	remaining: 1m 38s
162:	learn: 0.2490330	total: 19.3s	remaining: 1m 38s
163:	learn: 0.2490129	total: 19.4s	remaining: 1m 38s
164:	learn: 0.2489840	total: 19.5s	remaining: 1m 38s
165:	learn: 0.2489721	total: 19.6s	remaining: 1m 38s
166:	learn: 0.2489624	total: 19.7s	remaining: 1m 38s
167:	learn: 0.2489398	total: 19.8s	remaining: 1m 38s
168:	learn: 0.2489152	total: 19.9s	remaining: 1m 38s
169:	learn: 0.2488925	total: 20.1s	remaining: 1m 37s
170:	learn: 0.2488557	total: 20.2s	remaining: 1m 37s
171:	learn: 0.2488385	total: 20.3s	remaining: 1m 37s
172:	learn: 0.2487987	total: 20.4s	remaining: 1m 37s
173:	learn: 0.2487671	total: 20.6s	remaining: 1m 37s
174:	learn: 0.2487504	total: 20.7s	remaining: 1m

312:	learn: 0.2453079	total: 35.8s	remaining: 1m 18s
313:	learn: 0.2452779	total: 35.9s	remaining: 1m 18s
314:	learn: 0.2452468	total: 36s	remaining: 1m 18s
315:	learn: 0.2452245	total: 36.1s	remaining: 1m 18s
316:	learn: 0.2451983	total: 36.2s	remaining: 1m 18s
317:	learn: 0.2451701	total: 36.3s	remaining: 1m 17s
318:	learn: 0.2451618	total: 36.4s	remaining: 1m 17s
319:	learn: 0.2451269	total: 36.6s	remaining: 1m 17s
320:	learn: 0.2450976	total: 36.7s	remaining: 1m 17s
321:	learn: 0.2450767	total: 36.8s	remaining: 1m 17s
322:	learn: 0.2450612	total: 36.9s	remaining: 1m 17s
323:	learn: 0.2450391	total: 37s	remaining: 1m 17s
324:	learn: 0.2450123	total: 37.1s	remaining: 1m 17s
325:	learn: 0.2449928	total: 37.2s	remaining: 1m 17s
326:	learn: 0.2449693	total: 37.4s	remaining: 1m 16s
327:	learn: 0.2449524	total: 37.5s	remaining: 1m 16s
328:	learn: 0.2449296	total: 37.6s	remaining: 1m 16s
329:	learn: 0.2449153	total: 37.7s	remaining: 1m 16s
330:	learn: 0.2448929	total: 37.8s	remaining: 1m 1

470:	learn: 0.2419927	total: 52.3s	remaining: 58.7s
471:	learn: 0.2419726	total: 52.4s	remaining: 58.6s
472:	learn: 0.2419530	total: 52.5s	remaining: 58.5s
473:	learn: 0.2419432	total: 52.6s	remaining: 58.3s
474:	learn: 0.2419288	total: 52.7s	remaining: 58.2s
475:	learn: 0.2419042	total: 52.8s	remaining: 58.1s
476:	learn: 0.2418853	total: 52.9s	remaining: 58s
477:	learn: 0.2418596	total: 53s	remaining: 57.9s
478:	learn: 0.2418364	total: 53.1s	remaining: 57.7s
479:	learn: 0.2418149	total: 53.2s	remaining: 57.6s
480:	learn: 0.2418039	total: 53.3s	remaining: 57.5s
481:	learn: 0.2417876	total: 53.4s	remaining: 57.4s
482:	learn: 0.2417761	total: 53.5s	remaining: 57.3s
483:	learn: 0.2417638	total: 53.6s	remaining: 57.2s
484:	learn: 0.2417487	total: 53.7s	remaining: 57s
485:	learn: 0.2417320	total: 53.8s	remaining: 56.9s
486:	learn: 0.2417193	total: 53.9s	remaining: 56.8s
487:	learn: 0.2417119	total: 54s	remaining: 56.7s
488:	learn: 0.2416929	total: 54.1s	remaining: 56.6s
489:	learn: 0.241675

631:	learn: 0.2392615	total: 1m 9s	remaining: 40.4s
632:	learn: 0.2392452	total: 1m 9s	remaining: 40.3s
633:	learn: 0.2392338	total: 1m 9s	remaining: 40.1s
634:	learn: 0.2392160	total: 1m 9s	remaining: 40s
635:	learn: 0.2392008	total: 1m 9s	remaining: 39.9s
636:	learn: 0.2391880	total: 1m 9s	remaining: 39.8s
637:	learn: 0.2391683	total: 1m 9s	remaining: 39.7s
638:	learn: 0.2391609	total: 1m 10s	remaining: 39.6s
639:	learn: 0.2391396	total: 1m 10s	remaining: 39.5s
640:	learn: 0.2391260	total: 1m 10s	remaining: 39.4s
641:	learn: 0.2391042	total: 1m 10s	remaining: 39.3s
642:	learn: 0.2390934	total: 1m 10s	remaining: 39.1s
643:	learn: 0.2390663	total: 1m 10s	remaining: 39s
644:	learn: 0.2390454	total: 1m 10s	remaining: 38.9s
645:	learn: 0.2390268	total: 1m 10s	remaining: 38.8s
646:	learn: 0.2390011	total: 1m 10s	remaining: 38.7s
647:	learn: 0.2389796	total: 1m 11s	remaining: 38.6s
648:	learn: 0.2389610	total: 1m 11s	remaining: 38.5s
649:	learn: 0.2389540	total: 1m 11s	remaining: 38.4s
650:

788:	learn: 0.2365501	total: 1m 27s	remaining: 23.3s
789:	learn: 0.2365315	total: 1m 27s	remaining: 23.2s
790:	learn: 0.2365274	total: 1m 27s	remaining: 23.1s
791:	learn: 0.2365043	total: 1m 27s	remaining: 22.9s
792:	learn: 0.2364730	total: 1m 27s	remaining: 22.8s
793:	learn: 0.2364583	total: 1m 27s	remaining: 22.7s
794:	learn: 0.2364369	total: 1m 27s	remaining: 22.6s
795:	learn: 0.2364256	total: 1m 27s	remaining: 22.5s
796:	learn: 0.2364090	total: 1m 27s	remaining: 22.4s
797:	learn: 0.2363937	total: 1m 27s	remaining: 22.3s
798:	learn: 0.2363695	total: 1m 28s	remaining: 22.2s
799:	learn: 0.2363631	total: 1m 28s	remaining: 22s
800:	learn: 0.2363422	total: 1m 28s	remaining: 21.9s
801:	learn: 0.2363279	total: 1m 28s	remaining: 21.8s
802:	learn: 0.2363134	total: 1m 28s	remaining: 21.7s
803:	learn: 0.2362977	total: 1m 28s	remaining: 21.6s
804:	learn: 0.2362759	total: 1m 28s	remaining: 21.5s
805:	learn: 0.2362557	total: 1m 28s	remaining: 21.4s
806:	learn: 0.2362365	total: 1m 28s	remaining: 2

945:	learn: 0.2339484	total: 1m 42s	remaining: 5.88s
946:	learn: 0.2339318	total: 1m 43s	remaining: 5.77s
947:	learn: 0.2339133	total: 1m 43s	remaining: 5.66s
948:	learn: 0.2339045	total: 1m 43s	remaining: 5.55s
949:	learn: 0.2338887	total: 1m 43s	remaining: 5.44s
950:	learn: 0.2338741	total: 1m 43s	remaining: 5.33s
951:	learn: 0.2338614	total: 1m 43s	remaining: 5.22s
952:	learn: 0.2338446	total: 1m 43s	remaining: 5.11s
953:	learn: 0.2338301	total: 1m 43s	remaining: 5s
954:	learn: 0.2338057	total: 1m 43s	remaining: 4.89s
955:	learn: 0.2337905	total: 1m 43s	remaining: 4.78s
956:	learn: 0.2337663	total: 1m 44s	remaining: 4.67s
957:	learn: 0.2337482	total: 1m 44s	remaining: 4.56s
958:	learn: 0.2337288	total: 1m 44s	remaining: 4.46s
959:	learn: 0.2337175	total: 1m 44s	remaining: 4.34s
960:	learn: 0.2336980	total: 1m 44s	remaining: 4.24s
961:	learn: 0.2336868	total: 1m 44s	remaining: 4.13s
962:	learn: 0.2336694	total: 1m 44s	remaining: 4.02s
963:	learn: 0.2336516	total: 1m 44s	remaining: 3.

In [45]:
cb_pred = cbc.predict_proba(X_test[floats]).T[1]
roc_auc_score(y_test, cb_pred)

0.7462555703898908

__Задание 3. (2 балла)__

Подберите с CV=3 оптимальные параметры алгоритмов, изменяя:

* глубину деревьев;
* количество деревьев;
* темп обучения;
* оптимизируемый функционал.

Проанализируйте соотношения глубины и количества деревьев в зависимости от алгоритма.

In [51]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
gkf = KFold(n_splits=3, random_state=42).split(X=X_train[floats], y=y_train)

param_grid = {
    'max_depth': range(3, 16),
    'n_estimators ': [10, 20, 50, 100, 200, 300, 500, 1000],
    'learning_rate': np.logspace(-2, -0.2, 10),
    'eval_metric': ['mse', 'mae']
    }

lgbm_estimator = lgbm.LGBMClassifier()
gsearch = GridSearchCV(estimator=lgbm_estimator, param_grid=param_grid, cv=gkf)
lgbm_model = gsearch.fit(X=X_train[floats], y=y_train)

/home/tavplubix/.local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/tavplubix/.local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/tavplubix/.local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/tavplubix/.local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is a

/home/tavplubix/.local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/tavplubix/.local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/tavplubix/.local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/tavplubix/.local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is a

/home/tavplubix/.local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/tavplubix/.local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/tavplubix/.local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/tavplubix/.local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is a

/home/tavplubix/.local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/tavplubix/.local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/tavplubix/.local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/tavplubix/.local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is a

/home/tavplubix/.local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/tavplubix/.local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/tavplubix/.local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/tavplubix/.local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is a

...

/home/tavplubix/.local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/tavplubix/.local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/tavplubix/.local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/tavplubix/.local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is a

/home/tavplubix/.local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/tavplubix/.local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/tavplubix/.local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/tavplubix/.local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is a

/home/tavplubix/.local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/tavplubix/.local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/tavplubix/.local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/tavplubix/.local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is a

/home/tavplubix/.local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/tavplubix/.local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/tavplubix/.local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/tavplubix/.local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is a

KeyboardInterrupt: 

0.7442510162068767

In [33]:
np.unique(lgbm_pred)

array([0, 1])

__Задание 4. (3.5 балла)__

Добавьте категориальные признаки к вещественным следующими способами:

* как OHE признаки;
* как счетчики со сглаживанием.

При подсчете счетчиков запрещается использование циклов. 

На получившихся датасетах подберите параметры у каждого из алгоритмов. Как меняется время, необходимое для обучения модели в зависимости от способа кодирования? Сравните полученные результаты с встроенными методами обработки категориальных признаков. 

In [15]:
### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ

__Задание 5. (1 балл)__

Реализуйте блендинг подобранных в предыдущем задании моделей и сравните качество.

In [16]:
### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ

__Задание 6. (1.5 балла)__

В задании 3 вы подобрали гиперпараметры для LightGBM и CatBoost на вещественных признаках. Визуализируйте важности признаков, посчитанные этими алгоритмами, в виде горизонтального bar-plot (отсортируйте признаки по убыванию важности, подпишите названия признаков по оси y).

Для каждого из двух алгоритмов удалите неважные признаки (обычно по bar-plot хорошо видно порог на важность, с которого начинается хвост неважных признаков) и обучите ту же модель на получившихся данных. Сильно ли упало качество при удалении признаков, которые модель считает неважными?

In [17]:
### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ